In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import anndata
import scanpy as sc
from ALLCools.mcds import MCDS
from ALLCools.clustering import tsne, significant_pc_test, filter_regions, remove_black_list_region, lsi, binarize_matrix
from ALLCools.plot import *
from ALLCools.clustering import ConsensusClustering, Dendrogram, get_pc_centers

### this notebook generate .phy files used in Iqtree that construct lineage tree with methylation data.
### formate the .phy the to the following structure: 

find the tutorial here: http://www.iqtree.org/doc/Tutorial 

- the first number is the number of cells, the second is the length of sequence
- 1 means hypomethylation (from 5kb bin) - means gap (ignor gap bere)

4 6  
S1   010101   
S2   110011  
S3   0--100  
S4   10--10  

In [10]:
metadata = pd.read_csv('../metadata/GBM_metadata_9samples.csv', index_col = 0)
mcds = MCDS.open('/gale/netapp/entex/GBM/*/matrix/*.mcds', var_dim='chrom5k')

In [13]:
metadata = metadata.loc[metadata['Sample']=='CW_2209',]
mcds.add_cell_metadata(metadata)
selc = metadata.index
mcds = mcds.sel(cell = mcds.get_index('cell').intersection(selc))
mcds

<xarray.MCDS>
Dimensions:                    (cell: 1153, chrom5k: 617669)
Coordinates: (12/106)
  * cell                       (cell) object 'CW_2209_1-CW_2209_2-A10-AD001' ...
  * chrom5k                    (chrom5k) <U11 'chr1_0' 'chr1_1' ... 'chrY_11445'
    chrom5k_chrom              (chrom5k) <U5 dask.array<chunksize=(617669,), meta=np.ndarray>
    chrom5k_end                (chrom5k) int64 dask.array<chunksize=(617669,), meta=np.ndarray>
    chrom5k_start              (chrom5k) int64 dask.array<chunksize=(617669,), meta=np.ndarray>
    cell_Plate                 (cell) object 'CW_2209_1' ... 'CW_2209_4'
    ...                         ...
    cell_chr19_3c              (cell) float64 -0.5824 -1.224 ... -1.098 0.1056
    cell_chr20_3c              (cell) float64 -0.2391 -1.021 ... -0.9967 -0.9315
    cell_chrX_3c               (cell) float64 -0.3159 0.4379 ... 0.7702 0.4699
    cell_chr21_3c              (cell) float64 -2.297 -2.255 ... -2.302 -1.594
    cell_chr22_3c              (cell) float64 -2.286 -1.543 ... -1.988 -1.038
    cell_type                  (cell) object 'non-malignant' ... 'malignant'
Data variables:
    chrom5k_da_CGN-hypo-score  (cell, chrom5k) float16 dask.array<chunksize=(68, 617669), meta=np.ndarray>
Attributes:
    obs_dim:  cell
    var_dim:  chrom5k

In [14]:
mcds = mcds.remove_black_list_region(black_list_path='/gale/netapp/home/hanliu/ref/blacklist/hg38-blacklist.v2.bed.gz')
chrom_to_remove = ['chrL', 'chrM', 'chrX', 'chrY']
mcds = mcds.remove_chromosome( chrom_to_remove,'chrom5k')
mcds

45816 chrom5k features removed due to overlapping (bedtools intersect -f 0.2) with black list regions.
34491 chrom5k features in ['chrL', 'chrM', 'chrX', 'chrY'] removed.


<xarray.MCDS>
Dimensions:                    (cell: 1153, chrom5k: 537362)
Coordinates: (12/106)
  * cell                       (cell) object 'CW_2209_1-CW_2209_2-A10-AD001' ...
  * chrom5k                    (chrom5k) <U11 'chr1_159' ... 'chr9_27643'
    chrom5k_chrom              (chrom5k) <U5 dask.array<chunksize=(537362,), meta=np.ndarray>
    chrom5k_end                (chrom5k) int64 dask.array<chunksize=(537362,), meta=np.ndarray>
    chrom5k_start              (chrom5k) int64 dask.array<chunksize=(537362,), meta=np.ndarray>
    cell_Plate                 (cell) object 'CW_2209_1' ... 'CW_2209_4'
    ...                         ...
    cell_chr19_3c              (cell) float64 -0.5824 -1.224 ... -1.098 0.1056
    cell_chr20_3c              (cell) float64 -0.2391 -1.021 ... -0.9967 -0.9315
    cell_chrX_3c               (cell) float64 -0.3159 0.4379 ... 0.7702 0.4699
    cell_chr21_3c              (cell) float64 -2.297 -2.255 ... -2.302 -1.594
    cell_chr22_3c              (cell) float64 -2.286 -1.543 ... -1.988 -1.038
    cell_type                  (cell) object 'non-malignant' ... 'malignant'
Data variables:
    chrom5k_da_CGN-hypo-score  (cell, chrom5k) float16 dask.array<chunksize=(68, 537362), meta=np.ndarray>
Attributes:
    obs_dim:  cell
    var_dim:  chrom5k

In [34]:
df = mcds['chrom5k_da_CGN-hypo-score'].to_pandas()

In [37]:
df = df.round().astype(int)
df

chrom5k,chr1_159,chr1_160,chr1_161,chr1_162,chr1_163,chr1_164,chr1_165,chr1_166,chr1_167,chr1_168,...,chr9_27634,chr9_27635,chr9_27636,chr9_27637,chr9_27638,chr9_27639,chr9_27640,chr9_27641,chr9_27642,chr9_27643
cell,,,,,,,,,,,,,,,,,,,,,
CW_2209_1-CW_2209_2-A10-AD001,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CW_2209_1-CW_2209_2-A10-AD002,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
CW_2209_1-CW_2209_2-A10-AD004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CW_2209_1-CW_2209_2-A10-AD008,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CW_2209_1-CW_2209_2-A10-AD012,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CW_2209_3-CW_2209_4-H9-AD006,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CW_2209_3-CW_2209_4-H9-AD007,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CW_2209_3-CW_2209_4-H9-AD008,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
import glob
cells = glob.glob('../CNV_analysis/bedfiles/cw_2209/*')
cells

['../CNV_analysis/bedfiles/cw_2209/CW_2209_1-CW_2209_2-A10-AD008.bed.gz',
 '../CNV_analysis/bedfiles/cw_2209/CW_2209_1-CW_2209_2-A10-AD012.bed.gz',
 '../CNV_analysis/bedfiles/cw_2209/CW_2209_1-CW_2209_2-A11-AD001.bed.gz',
 '../CNV_analysis/bedfiles/cw_2209/CW_2209_1-CW_2209_2-B10-AD004.bed.gz',
 '../CNV_analysis/bedfiles/cw_2209/CW_2209_1-CW_2209_2-B10-AD007.bed.gz',
 '../CNV_analysis/bedfiles/cw_2209/CW_2209_1-CW_2209_2-B10-AD008.bed.gz',
 '../CNV_analysis/bedfiles/cw_2209/CW_2209_1-CW_2209_2-B10-AD010.bed.gz',
 '../CNV_analysis/bedfiles/cw_2209/CW_2209_1-CW_2209_2-B11-AD001.bed.gz',
 '../CNV_analysis/bedfiles/cw_2209/CW_2209_1-CW_2209_2-C10-AD002.bed.gz',
 '../CNV_analysis/bedfiles/cw_2209/CW_2209_1-CW_2209_2-C10-AD004.bed.gz',
 '../CNV_analysis/bedfiles/cw_2209/CW_2209_1-CW_2209_2-C10-AD007.bed.gz',
 '../CNV_analysis/bedfiles/cw_2209/CW_2209_1-CW_2209_2-C10-AD010.bed.gz',
 '../CNV_analysis/bedfiles/cw_2209/CW_2209_1-CW_2209_2-C10-AD012.bed.gz',
 '../CNV_analysis/bedfiles/cw_2209/CW_

In [41]:
cell_index = [ c.split('/')[-1].split('.')[0] for c in cells]

In [51]:
sum(~df.columns.str.contains('chr7|chr10|chr19', regex=True))

469227

In [ ]:
#remove chr7 10 19, which have cnv
df.loc[cell_index, ~df.columns.str.contains('chr7|chr10|chr19', regex=True)]

In [52]:
cell_df = df.loc[cell_index, ~df.columns.str.contains('chr7|chr10|chr19', regex=True)]
cell_df

chrom5k,chr1_159,chr1_160,chr1_161,chr1_162,chr1_163,chr1_164,chr1_165,chr1_166,chr1_167,chr1_168,...,chr9_27634,chr9_27635,chr9_27636,chr9_27637,chr9_27638,chr9_27639,chr9_27640,chr9_27641,chr9_27642,chr9_27643
cell,,,,,,,,,,,,,,,,,,,,,
CW_2209_1-CW_2209_2-A10-AD008,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CW_2209_1-CW_2209_2-A10-AD012,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CW_2209_1-CW_2209_2-A11-AD001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CW_2209_1-CW_2209_2-B10-AD004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
CW_2209_1-CW_2209_2-B10-AD007,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
CW_2209_1-CW_2209_2-B10-AD008,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
CW_2209_1-CW_2209_2-B10-AD010,0,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
CW_2209_1-CW_2209_2-B11-AD001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CW_2209_1-CW_2209_2-C10-AD002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0


In [53]:
cell_df = cell_df.astype(str)

In [46]:
''.join(cell_df.loc['CW_2209_1-CW_2209_2-A11-AD002',:].values[:50])

'00001000000010000000000000000100100000000100000000'

In [55]:
with open('cw_2209.phy', 'w') as f:
    f.write(str(len(cell_df.index)) + ' ' + str(len(cell_df.iloc[0,:])) + '\n')
    for i in cell_df.index:
        f.write(i+'\t')
        f.write(''.join(cell_df.loc[i,:].values))
        f.write('\n')
    f.close()

### example command line

`iqtree -s cw_2209.phy -st BIN -m TESTNEW -mset GTR2 -nt AUTO -bb 1000 -wbtl -abayes -alrt 1000 -nstop 500 -wsr -nm 2000 -v -alninfo`

`/iqtree-1.6.9-Linux/bin/iqtree -s ./my_sample_name.phy -st BIN -m TESTNEW -mset GTR2 -nt AUTO -bb 1000 -wbtl -abayes -alrt 1000 -nstop 500 -wsr -nm 2000 -v -alninfo`

`-s` is filet name
`-m` is the option to specify the model name to use during the analysis. 
We opted for the new model selection procedure64 (`-m TESTNEW`), which additionally implements the FreeRate heterogeneity model inferring the site rates directly from the data

`-mset GTR2` General time reversible model ‘GTR2’ consistently outperformed the other model tested (Jukes–Cantor type model) for our methylation binary data

`Q-TREE` can utilize multiple CPU cores to speed up the analysis. A complement option -T (or -nt for version 1.x) allows specifying the number of CPU cores to use. 

set an upper limit of CPU cores that can automatically be assigned. Use the -ntmax option to do so

`-bb` or `-B` specifies the number of bootstrap replicates where 1000 is the minimum number recommended. # for version 1.x change -B to -bb

`-alrt` specifies the number of bootstrap replicates for SH-aLRT where 1000 is the minimum number recommended.

IQ-TREE also supports other tests such as the aBayes test 

read more: http://www.iqtree.org/doc/Tutorial 

In [69]:
tree.ntips

62

In [70]:
#tree.get_tip_labels()
#tree.nnodes
types = ['cancer']*40 +['normal']*(tree.ntips - 40)
colorlist = ["#d6557c" if "normal" in tip else "#5384a3" for tip in types]
tree.draw(
    tip_labels_align=True,
    tip_labels_colors=colorlist
);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="1000.0px" viewBox="0 0 500.0 1000.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t2407ff3e71e54749938e0b751c6b9289"> MGH124_CD45_P7_G1_S4_L003.CCATAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_C1_S2_L003.AGGATG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_E1_S3_L003.TGCTGC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_G1_S4_L003.ACAACC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_G1_S4_L003.TGCTGC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_A1_S1_L003.ACTCAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_A1_S1_L003.CATGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_G1_S4_L003.TCTCTG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_A1_S1_L003.AGAAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_A1_S1_L003.GCATTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_C1_S2_L003.ACAACC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_C1_S2_L003.CATGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_A1_S1_L003.CGGTAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_G1_S4_L003.CGGTAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_E1_S3_L003.CTCAGC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_C1_S2_L003.TGTAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_C1_S2_L003.TGCTGC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_G1_S4_L003.GTTGAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_A1_S1_L003.GTGAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_G1_S4_L003.TGTAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_E1_S3_L003.CATGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_E1_S3_L003.GCATTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_C1_S2_L003.CCATAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_C1_S2_L003.TATCTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_C1_S2_L003.CAAGAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_A1_S1_L003.GAAGTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_G1_S4_L003.CTATTG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_G1_S4_L003.GTGAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_G1_S4_L003.GCATTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_G1_S4_L003.CCTTCG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_E1_S3_L003.TGTAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_A1_S1_L003.TCTCTG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_E1_S3_L003.CGGTAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_E1_S3_L003.GTGAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_E1_S3_L003.GTTGAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_E1_S3_L003.GGTAAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_E1_S3_L003.ACAGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_A1_S1_L003.CTATTG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_E1_S3_L003.CCTTCG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_G1_S4_L003.CATGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_E1_S3_L003.ACTCAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_G1_S4_L003.ACTCAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_E1_S3_L003.ACAACC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_A1_S1_L003.TGACAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_C1_S2_L003.CCTTCG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_E1_S3_L003.CCATAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_E1_S3_L003.CTATTG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_E1_S3_L003.ATCAAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_C1_S2_L003.CTATTG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_G1_S4_L003.AGAAGG.R1.fastq_bismark_bt2_pe.bismark.cov M

## replace the tree with iqtree generated phylogeny tree (exclude cnv chrm)

replace the color scheme with cell type, clusters, or cnv inference

In [119]:
import toytree
#cw_2209
newick = ''
with open('cw_2209.phy.parstree', 'r') as f:
    newick = f.read()

tree = toytree.tree(newick)
canvas, axes, mark = tree.draw(use_edge_lengths=False)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="395.0px" height="900.0px" viewBox="0 0 395.0 900.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t4f4547c372964f3c9663ff51753c7465"> CW_2209_1-CW_2209_2-D10-AD006 CW_2209_1-CW_2209_2-B10-AD002 CW_2209_1-CW_2209_2-E10-AD008 CW_2209_1-CW_2209_2-H10-AD007 CW_2209_1-CW_2209_2-D10-AD012 CW_2209_1-CW_2209_2-A10-AD008 CW_2209_1-CW_2209_2-H10-AD001 CW_2209_1-CW_2209_2-A11-AD004 CW_2209_1-CW_2209_2-A11-AD010 CW_2209_1-CW_2209_2-F10-AD001 CW_2209_1-CW_2209_2-B11-AD001 CW_2209_1-CW_2209_2-A10-AD012 CW_2209_1-CW_2209_2-H10-AD012 CW_2209_1-CW_2209_2-B11-AD002 CW_2209_1-CW_2209_2-F10-AD012 CW_2209_1-CW_2209_2-A11-AD002 CW_2209_1-CW_2209_2-B10-AD007 CW_2209_1-CW_2209_2-C10-AD002 CW_2209_1-CW_2209_2-D10-AD001 CW_2209_1-CW_2209_2-B10-AD004 CW_2209_1-CW_2209_2-A10-AD004 CW_2209_1-CW_2209_2-A11-AD006 CW_2209_1-CW_2209_2-A11-AD007 CW_2209_1-CW_2209_2-C10-AD012 CW_2209_1-CW_2209_2-F10-AD002 CW_2209_1-CW_2209_2-C10-AD007 CW_2209_1-CW_2209_2-H10-AD006 CW_2209_1-CW_2209_2-H10-AD004 CW_2209_1-CW_2209_2-G10-AD010 CW_2209_1-CW_2209_2-H10-AD008 CW_2209_1-CW_2209_2-D10-AD007 CW_2209_1-CW_2209_2-A11-AD012 CW_2209_1-CW_2209_2-C10-AD004 CW_2209_1-CW_2209_2-C10-AD010 CW_2209_1-CW_2209_2-F10-AD007 CW_2209_1-CW_2209_2-A11-AD008 CW_2209_1-CW_2209_2-D10-AD002 CW_2209_1-CW_2209_2-H10-AD010 CW_2209_1-CW_2209_2-E10-AD010 CW_2209_1-CW_2209_2-E10-AD006 CW_2209_1-CW_2209_2-E10-AD002 CW_2209_1-CW_2209_2-E10-AD007 CW_2209_1-CW_2209_2-F10-AD006 CW_2209_1-CW_2209_2-A11-AD001 CW_2209_1-CW_2209_2-C10-AD001 CW_2209_1-CW_2209_2-F10-AD008 CW_2209_1-CW_2209_2-B10-AD010 CW_2209_1-CW_2209_2-B10-AD008 CW_2209_1-CW_2209_2-G10-AD002 CW_2209_1-CW_2209_2-G10-AD006

In [124]:
cell_cnv = pd.read_csv('Newick_tree/test/CNV_chr_percell.tsv', sep = '\t', header = None,
            names = ['chrom','start','end','cell','cnv'])
cell_cnv

,chrom,start,end,cell,cnv
0,chr1,1,2345598,CW_2209_1.CW_2209_2.E10.AD010,1
1,chr1,1,4112610,CW_2209_1.CW_2209_2.H10.AD004,1
2,chr1,2345599,6148783,CW_2209_1.CW_2209_2.A11.AD006,-1
3,chr1,4112611,6661103,CW_2209_1.CW_2209_2.H10.AD004,-1
4,chr1,1,7681749,CW_2209_1.CW_2209_2.B11.AD002,-1
...,...,...,...,...,...
1433,chrY,1,59373566,CW_2209_1.CW_2209_2.G10.AD006,-1
1434,chrY,1,59373566,CW_2209_1.CW_2209_2.G10.AD010,-1
1435,chrY,1,59373566,CW_2209_1.CW_2209_2.H10.AD001,-1
1436,chrY,1,59373566,CW_2209_1.CW_2209_2.H10.AD004,-1


In [130]:
cell_cnv['cell'] = cell_cnv['cell'].str.replace('.','-')

/tmp/ipykernel_18143/2414418508.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  cell_cnv['cell'] = cell_cnv['cell'].str.replace('.','-')


In [131]:
chr7 = cell_cnv.loc[cell_cnv['chrom']=='chr7',:]
# index of -1, index of 1, intersection of both: del_dup
# only -1: del, only 1: dup, none: normal 
g = chr7.groupby(by='cnv')
gain = set(g.get_group(1)['cell'])
loss = set(g.get_group(-1)['cell'])
both = gain.intersection(loss)
chr7_cnv = {'gain':gain-both, 'loss': loss-both, 'both': both, 'normal': set(tree.get_tip_labels())-gain - loss}
chr7_cnv

 #[print(x, y) for x in iter1 for y in iter2] 
chr7_cnv = { cell: cnv for cnv, l in chr7_cnv.items() for cell in l}

In [136]:
#colorlist
#set(tree.get_tip_labels())-gain - loss
chr7_cnv

{'CW_2209_1-CW_2209_2-C10-AD001': 'gain',
 'CW_2209_1-CW_2209_2-H10-AD004': 'gain',
 'CW_2209_1-CW_2209_2-A10-AD008': 'gain',
 'CW_2209_1-CW_2209_2-B10-AD004': 'gain',
 'CW_2209_1-CW_2209_2-E10-AD008': 'gain',
 'CW_2209_1-CW_2209_2-A11-AD012': 'gain',
 'CW_2209_1-CW_2209_2-E10-AD006': 'gain',
 'CW_2209_1-CW_2209_2-D10-AD007': 'gain',
 'CW_2209_1-CW_2209_2-D10-AD012': 'gain',
 'CW_2209_1-CW_2209_2-A11-AD007': 'gain',
 'CW_2209_1-CW_2209_2-B10-AD008': 'gain',
 'CW_2209_1-CW_2209_2-C10-AD010': 'gain',
 'CW_2209_1-CW_2209_2-G10-AD002': 'gain',
 'CW_2209_1-CW_2209_2-F10-AD001': 'gain',
 'CW_2209_1-CW_2209_2-A11-AD001': 'gain',
 'CW_2209_1-CW_2209_2-A11-AD008': 'gain',
 'CW_2209_1-CW_2209_2-E10-AD007': 'gain',
 'CW_2209_1-CW_2209_2-G10-AD006': 'gain',
 'CW_2209_1-CW_2209_2-G10-AD010': 'gain',
 'CW_2209_1-CW_2209_2-C10-AD004': 'gain',
 'CW_2209_1-CW_2209_2-D10-AD002': 'gain',
 'CW_2209_1-CW_2209_2-B10-AD010': 'gain',
 'CW_2209_1-CW_2209_2-E10-AD002': 'gain',
 'CW_2209_1-CW_2209_2-A10-AD012': 

In [137]:
color_code = {'both':'mediumseagreen','loss':"#d6557c", 'gain':"#5384a3",'normal':'black'}
colorlist = [color_code[chr7_cnv[tip]] for tip in tree.get_tip_labels()]
tree.draw(
    #tip_labels_align=True,
    tip_labels_colors=colorlist,
    use_edge_lengths=False
);
#gain: blue, loss: red:  both: green 

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="395.0px" height="900.0px" viewBox="0 0 395.0 900.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t3c2a9510dade497f928193710b458513"> CW_2209_1-CW_2209_2-D10-AD006 CW_2209_1-CW_2209_2-B10-AD002 CW_2209_1-CW_2209_2-E10-AD008 CW_2209_1-CW_2209_2-H10-AD007 CW_2209_1-CW_2209_2-D10-AD012 CW_2209_1-CW_2209_2-A10-AD008 CW_2209_1-CW_2209_2-H10-AD001 CW_2209_1-CW_2209_2-A11-AD004 CW_2209_1-CW_2209_2-A11-AD010 CW_2209_1-CW_2209_2-F10-AD001 CW_2209_1-CW_2209_2-B11-AD001 CW_2209_1-CW_2209_2-A10-AD012 CW_2209_1-CW_2209_2-H10-AD012 CW_2209_1-CW_2209_2-B11-AD002 CW_2209_1-CW_2209_2-F10-AD012 CW_2209_1-CW_2209_2-A11-AD002 CW_2209_1-CW_2209_2-B10-AD007 CW_2209_1-CW_2209_2-C10-AD002 CW_2209_1-CW_2209_2-D10-AD001 CW_2209_1-CW_2209_2-B10-AD004 CW_2209_1-CW_2209_2-A10-AD004 CW_2209_1-CW_2209_2-A11-AD006 CW_2209_1-CW_2209_2-A11-AD007 CW_2209_1-CW_2209_2-C10-AD012 CW_2209_1-CW_2209_2-F10-AD002 CW_2209_1-CW_2209_2-C10-AD007 CW_2209_1-CW_2209_2-H10-AD006 CW_2209_1-CW_2209_2-H10-AD004 CW_2209_1-CW_2209_2-G10-AD010 CW_2209_1-CW_2209_2-H10-AD008 CW_2209_1-CW_2209_2-D10-AD007 CW_2209_1-CW_2209_2-A11-AD012 CW_2209_1-CW_2209_2-C10-AD004 CW_2209_1-CW_2209_2-C10-AD010 CW_2209_1-CW_2209_2-F10-AD007 CW_2209_1-CW_2209_2-A11-AD008 CW_2209_1-CW_2209_2-D10-AD002 CW_2209_1-CW_2209_2-H10-AD010 CW_2209_1-CW_2209_2-E10-AD010 CW_2209_1-CW_2209_2-E10-AD006 CW_2209_1-CW_2209_2-E10-AD002 CW_2209_1-CW_2209_2-E10-AD007 CW_2209_1-CW_2209_2-F10-AD006 CW_2209_1-CW_2209_2-A11-AD001 CW_2209_1-CW_2209_2-C10-AD001 CW_2209_1-CW_2209_2-F10-AD008 CW_2209_1-CW_2209_2-B10-AD010 CW_2209_1-CW_2209_2-B10-AD008 CW_2209_1-CW_2209_2-G10-AD002 CW_2209_1-CW_2209_2-G10-AD006

In [138]:
chr10 = cell_cnv.loc[cell_cnv['chrom']=='chr10',:]
# index of -1, index of 1, intersection of both: del_dup
# only -1: del, only 1: dup, none: normal 
g = chr10.groupby(by='cnv')
gain = set(g.get_group(1)['cell'])
loss = set(g.get_group(-1)['cell'])
both = gain.intersection(loss)
chr10_cnv = {'gain':gain-both, 'loss': loss-both, 'both': both, 'normal': set(tree.get_tip_labels())-gain - loss}
 #[print(x, y) for x in iter1 for y in iter2] 
chr10_cnv = { cell: cnv for cnv, l in chr10_cnv.items() for cell in l}

In [139]:
color_code = {'both':'mediumseagreen','loss':"#d6557c", 'gain':"#5384a3",'normal':'black'}
colorlist = [color_code[chr10_cnv[tip]] for tip in tree.get_tip_labels()]
tree.draw(
    #tip_labels_align=True,
    tip_labels_colors=colorlist,
    use_edge_lengths=False
);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="395.0px" height="900.0px" viewBox="0 0 395.0 900.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t032b85ef2e1047e7a2cc5fa0fe86503c"> CW_2209_1-CW_2209_2-D10-AD006 CW_2209_1-CW_2209_2-B10-AD002 CW_2209_1-CW_2209_2-E10-AD008 CW_2209_1-CW_2209_2-H10-AD007 CW_2209_1-CW_2209_2-D10-AD012 CW_2209_1-CW_2209_2-A10-AD008 CW_2209_1-CW_2209_2-H10-AD001 CW_2209_1-CW_2209_2-A11-AD004 CW_2209_1-CW_2209_2-A11-AD010 CW_2209_1-CW_2209_2-F10-AD001 CW_2209_1-CW_2209_2-B11-AD001 CW_2209_1-CW_2209_2-A10-AD012 CW_2209_1-CW_2209_2-H10-AD012 CW_2209_1-CW_2209_2-B11-AD002 CW_2209_1-CW_2209_2-F10-AD012 CW_2209_1-CW_2209_2-A11-AD002 CW_2209_1-CW_2209_2-B10-AD007 CW_2209_1-CW_2209_2-C10-AD002 CW_2209_1-CW_2209_2-D10-AD001 CW_2209_1-CW_2209_2-B10-AD004 CW_2209_1-CW_2209_2-A10-AD004 CW_2209_1-CW_2209_2-A11-AD006 CW_2209_1-CW_2209_2-A11-AD007 CW_2209_1-CW_2209_2-C10-AD012 CW_2209_1-CW_2209_2-F10-AD002 CW_2209_1-CW_2209_2-C10-AD007 CW_2209_1-CW_2209_2-H10-AD006 CW_2209_1-CW_2209_2-H10-AD004 CW_2209_1-CW_2209_2-G10-AD010 CW_2209_1-CW_2209_2-H10-AD008 CW_2209_1-CW_2209_2-D10-AD007 CW_2209_1-CW_2209_2-A11-AD012 CW_2209_1-CW_2209_2-C10-AD004 CW_2209_1-CW_2209_2-C10-AD010 CW_2209_1-CW_2209_2-F10-AD007 CW_2209_1-CW_2209_2-A11-AD008 CW_2209_1-CW_2209_2-D10-AD002 CW_2209_1-CW_2209_2-H10-AD010 CW_2209_1-CW_2209_2-E10-AD010 CW_2209_1-CW_2209_2-E10-AD006 CW_2209_1-CW_2209_2-E10-AD002 CW_2209_1-CW_2209_2-E10-AD007 CW_2209_1-CW_2209_2-F10-AD006 CW_2209_1-CW_2209_2-A11-AD001 CW_2209_1-CW_2209_2-C10-AD001 CW_2209_1-CW_2209_2-F10-AD008 CW_2209_1-CW_2209_2-B10-AD010 CW_2209_1-CW_2209_2-B10-AD008 CW_2209_1-CW_2209_2-G10-AD002 CW_2209_1-CW_2209_2-G10-AD006

In [141]:
chr6 = cell_cnv.loc[cell_cnv['chrom']=='chr19',:]
# index of -1, index of 1, intersection of both: del_dup
# only -1: del, only 1: dup, none: normal 
g = chr6.groupby(by='cnv')
gain = set(g.get_group(1)['cell'])
loss = set(g.get_group(-1)['cell'])
both = gain.intersection(loss)
chr6_cnv = {'gain':gain-both, 'loss': loss-both, 'both': both, 'normal': set(tree.get_tip_labels())-gain - loss}
 #[print(x, y) for x in iter1 for y in iter2] 
chr6_cnv = { cell: cnv for cnv, l in chr6_cnv.items() for cell in l}

color_code = {'both':'mediumseagreen','loss':"#d6557c", 'gain':"#5384a3",'normal':'black'}
colorlist = [color_code[chr6_cnv[tip]] for tip in tree.get_tip_labels()]
tree.draw(
    #tip_labels_align=True,
    tip_labels_colors=colorlist,
    use_edge_lengths=False
);
#gain: blue, loss: red:  both: green 

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="395.0px" height="900.0px" viewBox="0 0 395.0 900.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tf9709fa2964a4da19b067900740cb47d"> CW_2209_1-CW_2209_2-D10-AD006 CW_2209_1-CW_2209_2-B10-AD002 CW_2209_1-CW_2209_2-E10-AD008 CW_2209_1-CW_2209_2-H10-AD007 CW_2209_1-CW_2209_2-D10-AD012 CW_2209_1-CW_2209_2-A10-AD008 CW_2209_1-CW_2209_2-H10-AD001 CW_2209_1-CW_2209_2-A11-AD004 CW_2209_1-CW_2209_2-A11-AD010 CW_2209_1-CW_2209_2-F10-AD001 CW_2209_1-CW_2209_2-B11-AD001 CW_2209_1-CW_2209_2-A10-AD012 CW_2209_1-CW_2209_2-H10-AD012 CW_2209_1-CW_2209_2-B11-AD002 CW_2209_1-CW_2209_2-F10-AD012 CW_2209_1-CW_2209_2-A11-AD002 CW_2209_1-CW_2209_2-B10-AD007 CW_2209_1-CW_2209_2-C10-AD002 CW_2209_1-CW_2209_2-D10-AD001 CW_2209_1-CW_2209_2-B10-AD004 CW_2209_1-CW_2209_2-A10-AD004 CW_2209_1-CW_2209_2-A11-AD006 CW_2209_1-CW_2209_2-A11-AD007 CW_2209_1-CW_2209_2-C10-AD012 CW_2209_1-CW_2209_2-F10-AD002 CW_2209_1-CW_2209_2-C10-AD007 CW_2209_1-CW_2209_2-H10-AD006 CW_2209_1-CW_2209_2-H10-AD004 CW_2209_1-CW_2209_2-G10-AD010 CW_2209_1-CW_2209_2-H10-AD008 CW_2209_1-CW_2209_2-D10-AD007 CW_2209_1-CW_2209_2-A11-AD012 CW_2209_1-CW_2209_2-C10-AD004 CW_2209_1-CW_2209_2-C10-AD010 CW_2209_1-CW_2209_2-F10-AD007 CW_2209_1-CW_2209_2-A11-AD008 CW_2209_1-CW_2209_2-D10-AD002 CW_2209_1-CW_2209_2-H10-AD010 CW_2209_1-CW_2209_2-E10-AD010 CW_2209_1-CW_2209_2-E10-AD006 CW_2209_1-CW_2209_2-E10-AD002 CW_2209_1-CW_2209_2-E10-AD007 CW_2209_1-CW_2209_2-F10-AD006 CW_2209_1-CW_2209_2-A11-AD001 CW_2209_1-CW_2209_2-C10-AD001 CW_2209_1-CW_2209_2-F10-AD008 CW_2209_1-CW_2209_2-B10-AD010 CW_2209_1-CW_2209_2-B10-AD008 CW_2209_1-CW_2209_2-G10-AD002 CW_2209_1-CW_2209_2-G10-AD006